In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objs as go

from scipy.optimize import least_squares # least squares non linear
from scipy.optimize import NonlinearConstraint
from scipy.optimize import minimize

from scipy.integrate import quad
from scipy import real

In [2]:
df = pd.read_csv('data.csv')

In [3]:
# FONCTION CHARACTERISTIQUE ln(ST) DE HESTON

def chfun_heston(s0, v0, vbar, a, vvol, r, rho, t, w):
    """
    Heston characteristic function.
    Inputs:
    s0: stock price
    v0: initial volatility (v0^2 initial variance)
    vbar: long-term variance mean
    a: variance mean-reversion speed
    vvol: volatility of the variance process
    r : risk-free rate
    rho: correlation between the Weiner processes for the stock price and its variance
    t: time
    w: points at which to evaluate the function
    Output:
    y: Characteristic function of log (St) in the Heston model
    """
    # Interim calculations
    alpha = -w*w/2 - 1j*w/2
    beta = a - rho*vvol*1j*w
    gamma = vvol*vvol/2
    h = np.sqrt(beta*beta - 4*alpha*gamma)
    rplus = (beta + h)/(vvol*vvol)
    rminus = (beta - h)/(vvol*vvol)
    g = rminus / rplus

    # Required inputs for the characteristic function
    C = a * (rminus * t - (2 / (vvol**2)) * np.log((1 - g * np.exp(-h*t))/(1 - g)))
    D = rminus * (1 - np.exp(-h * t))/(1 - g * np.exp(-h*t))

    # Characteristic function evaluated at points w
    y = np.exp(C*vbar + D*v0 + 1j*w*np.log(s0*np.exp(r*t)))
    return y


In [4]:
# POUR AVOIR LA FORMULE DU CALL

def call_heston_cf(s0, v0, vbar, a, vvol, r, rho, t, k, chfun_heston):
    """
    Heston call value using characteristic functions.
    Inputs:
    s0: stock price
    v0: initial volatility (v0^2 initial variance)
    vbar: long-term variance mean
    a: variance mean-reversion speed
    vvol: volatility of the variance process
    r: risk-free rate
    rho: correlation between the Weiner processes of the stock price and its variance
    t: time to maturity
    k: option strike
    chfun_heston: Heston characteristic function
    Output:
    y: Heston call value
    """
    # Inner integral 1
    def int1(w, s0, v0, vbar, a, vvol, r, rho, t, k):
        return np.real(np.exp(-1j * w * np.log(k)) * chfun_heston(s0, v0, vbar, a, vvol, r, rho, t, w - 1j) /
                       (1j * w * chfun_heston(s0, v0, vbar, a, vvol, r, rho, t, -1j)))
    
    int1_result, _ = quad(lambda w: int1(w, s0, v0, vbar, a, vvol, r, rho, t, k), 0, 100)
    pi1 = int1_result / np.pi + 0.5

    # Inner integral 2
    def int2(w, s0, v0, vbar, a, vvol, r, rho, t, k):
        return np.real(np.exp(-1j * w * np.log(k)) * chfun_heston(s0, v0, vbar, a, vvol, r, rho, t, w) / (1j * w))
    
    int2_result, _ = quad(lambda w: int2(w, s0, v0, vbar, a, vvol, r, rho, t, k), 0, 100)
    pi2 = int2_result / np.pi + 0.5

    # Calculate call value
    y = s0 * pi1 - np.exp(-r * t) * k * pi2
    return y